In [ ]:
#r "nuget:System.Text.Json, 8.0.1"
#r "nuget:Azure.AI.OpenAI, 1.0.0-beta.16"
#r "nuget:Azure.Identity, 1.10.4"
#r "nuget:DotNetEnv, 3.0.0"

In [ ]:
using System.Net;
using System.Net.Http;
using System.Text.Json.Nodes;
using System.Text.Json;
using System.IO; 

using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using DotNetEnv;

In [ ]:
Env.Load("../../../.env");

string openAIEndpoint = Environment.GetEnvironmentVariable("COMPLETIONS_OPENAI_ENDPOINT");
string openAIModelDeployment = Environment.GetEnvironmentVariable("COMPLETIONS_OPENAI_COMPLETION_MODEL_DEPLOYMENT");
string openAIApiVersion = "2024-03-01-preview";

var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions { 
    ExcludeEnvironmentCredential = true,
    ExcludeManagedIdentityCredential = true,
    ExcludeSharedTokenCacheCredential = true,
    ExcludeInteractiveBrowserCredential = true,
    ExcludeAzurePowerShellCredential = true,
    ExcludeVisualStudioCodeCredential = false,
    ExcludeAzureCliCredential = false
});

var openAIClient = new OpenAIClient(new Uri(openAIEndpoint), credential);


In [ ]:
var jsonStructure = new {
    classification = "",
    text = ""
};

var jsonStructureList = new List<object>();
jsonStructureList.Add(jsonStructure);

ChatCompletionsOptions options = new ChatCompletionsOptions()
{
    DeploymentName = openAIModelDeployment,
    MaxTokens = 4096,
    Temperature = 0.1f,
    NucleusSamplingFactor = 0.1f,
    EnableLogProbabilities = true,
    LogProbabilitiesPerToken = 3
};

options.Messages.Add(new ChatRequestSystemMessage($"You are an AI assistant that splits text, classifies it into categories, and returns the response as structured JSON objects using the following structure: {JsonSerializer.Serialize(jsonStructureList)}. Do not return as a code block."));
options.Messages.Add(new ChatRequestUserMessage("Classify the following text in the following categories: 'top_priority', 'high_priority', 'medium_priority', 'low_priority'."));
options.Messages.Add(new ChatRequestUserMessage(@"
Dear Team,

Thank you for attending our recent meeting on building an AI solution using Azure. I wanted to summarize the key points and action items discussed during the meeting:

1. Project Overview:
  - We discussed the need for an AI solution to enhance customer interactions.
  - The goal is to integrate intelligence into our communications across various channels.
2. Key Takeaways:
  - Azure Communication Services and Azure AI can be used together to automate and transform customer interactions.
  - Our solution will provide faster and informed responses across text-based bots, voice channels, email, and SMS.
  - Agents can leverage AI context to handle escalations effectively.
3. Action Items:
  - Set up a project team to explore Azure Communication Services and Azure AI.
  - Identify communication channels (e.g., website chat, phone, SMS) for integration.
  - Develop a proof-of-concept bot using ChatGPT and Azure OpenAI Service.
  - Create a roadmap for implementing the solution.
4. Next Steps:
  - Let's schedule a follow-up meeting next week to review progress and assign responsibilities.
  - Let's grab a coffee and have a chat.

Feel free to reach out if you have any questions or need further clarification. Looking forward to working together on this exciting project!

Best regards, 
James"));

var response = await openAIClient.GetChatCompletionsAsync(options);

var content = response.Value.Choices[0].Message.Content;

File.WriteAllText("Playground-dotnet-SDK-Response.json", JsonSerializer.Serialize(response.Value, new JsonSerializerOptions { WriteIndented = true }));
Console.WriteLine("Playground-dotnet-SDK-Response.json has been created with the response from the OpenAI API.");

File.WriteAllText("Playground-dotnet-SDK-ContentResponse.json", content);
Console.WriteLine("Playground-dotnet-SDK-ContentResponse.json has been created with the content from the response from the OpenAI API.");